<a href="https://colab.research.google.com/github/Chirag314/stock-market-zoomcamp/blob/main/02HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0) Imports and Installs

In [28]:
!pip install yfinance

In [29]:
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# 1) [Code Snippet 1] IPOs data from Web

* learn dataframes "stacking" (UNION ALL)
* learn about type casting
* generate new columns (simple and with function)


In [30]:
import pandas as pd
import requests
from io import StringIO

def get_ipos_by_year() -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [31]:
ipos_withdrawn = get_ipos_by_year()
ipos_withdrawn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [32]:
ipos_withdrawn.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [33]:
ipos_withdrawn["Company Name"].value_counts()

,count
Company Name,
"Odyssey Therapeutics, Inc.",1
"Unifoil Holdings, Inc.",1
"Aurion Biotech, Inc.",1
"PHI Group, Inc.",1
One Power Company,1
...,...
"Freehold Properties, Inc.",1
Chobani Inc.,1
iFIT Health & Fitness Inc.,1


In [34]:
def categorize_company_class(company_name):
    """Categorizes company names based on predefined patterns."""
    company_name = str(company_name) # Ensure the input is a string
    if "Acquisition Corp" in company_name or "Acquisition Corporation" in company_name:
        return "Acq.Corp"
    elif "Inc" in company_name or "Incorporated" in company_name:
        return "Inc"
    elif "Group" in company_name:
        return "Group"
    elif "Ltd" in company_name or "Limited" in company_name:
        return "Limited"
    elif "Holdings" in company_name:
        return "Holdings"
    else:
        return "Other"

# Apply the function to create the new 'Company Class' column
ipos_withdrawn['Company Class'] = ipos_withdrawn['Company Name'].apply(categorize_company_class)

# Display the value counts of the new column to check the distribution
display(ipos_withdrawn['Company Class'].value_counts())

,count
Company Class,
Inc,51
Acq.Corp,21
Limited,17
Other,6
Group,4
Holdings,1


In [35]:
def parse_price_range(price_range):
    """Parses the 'Price Range' string and returns the average price."""
    if price_range == '-':
        return None
    elif '-' in price_range:
        prices = price_range.replace('$', '').split('-')
        try:
            # Convert to float and calculate the average
            avg_price = (float(prices[0]) + float(prices[1])) / 2
            return avg_price
        except ValueError:
            return None  # Handle cases where conversion to float fails
    else:
        try:
            # Handle single price values
            return float(price_range.replace('$', ''))
        except ValueError:
            return None # Handle cases where conversion to float fails

# Apply the function to create the new 'Avg. price' column
ipos_withdrawn['Avg. price'] = ipos_withdrawn['Price Range'].apply(parse_price_range)

# Display the first few rows with the new column and its info
display(ipos_withdrawn.head())
ipos_withdrawn.info()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          100 non-null    object 
 1   Company Name    100 non-null    object 
 2   Price Range     100 non-null    object 
 3   Shares Offered  100 non-null    object 
 4   Company Class   100 non-null    object 
 5   Avg. price      73 non-null     float64
dtypes: float64(1), object(5)
memory usage: 4.8+ KB


In [36]:
# Convert 'Shares Offered' to numeric, coercing errors to NaN
ipos_withdrawn['Shares Offered'] = pd.to_numeric(ipos_withdrawn['Shares Offered'].str.replace(',', '').replace('-', ''), errors='coerce')

# Calculate 'Withdrawn Value'
ipos_withdrawn['Withdrawn Value'] = ipos_withdrawn['Shares Offered'] * ipos_withdrawn['Avg. price']

# Display the first few rows with the new column and its info
display(ipos_withdrawn.head())
ipos_withdrawn.info()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN,NaN
4,ONE,One Power Company,-,NaN,Other,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Symbol           100 non-null    object 
 1   Company Name     100 non-null    object 
 2   Price Range      100 non-null    object 
 3   Shares Offered   72 non-null     float64
 4   Company Class    100 non-null    object 
 5   Avg. price       73 non-null     float64
 6   Withdrawn Value  71 non-null     float64
dtypes: float64(3), object(4)
memory usage: 5.6+ KB


In [37]:
# Group by 'Company Class' and calculate the sum of 'Withdrawn Value'
total_withdrawn_value_by_class = ipos_withdrawn.groupby('Company Class')['Withdrawn Value'].sum()

# Find the class with the highest total withdrawn value
highest_value_class = total_withdrawn_value_by_class.idxmax()
highest_value = total_withdrawn_value_by_class.max()

print(f"The class with the highest total value of withdrawals is '{highest_value_class}' with a total value of {highest_value:.2f}")

# Display the total withdrawn value for all classes
display(total_withdrawn_value_by_class)

The class with the highest total value of withdrawals is 'Acq.Corp' with a total value of 4021000000.00


,Withdrawn Value
Company Class,
Acq.Corp,4.021000e+09
Group,3.378750e+07
Holdings,7.500000e+07
Inc,2.257164e+09
Limited,5.497346e+08
Other,7.679200e+08


Code for question 2

In [38]:
def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

# Get 2024 IPO data
ipos_2024 = get_ipos_by_year(2024)

# Convert 'IPO Date' to datetime objects
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'])

# Filter for IPOs before June 1, 2024
ipos_2024_filtered = ipos_2024[ipos_2024['IPO Date'] < '2024-06-01']

# Display the number of tickers
print(f"Number of tickers after filtering: {ipos_2024_filtered['Symbol'].nunique()}")

# Display the head of the filtered DataFrame
display(ipos_2024_filtered.head())

Number of tickers after filtering: 77


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$14.82,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.27,113.35%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$19.91,1.79%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.53,5.30%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.30,-92.47%


In [39]:
all_tickers = ipos_2024_filtered['Symbol'].tolist()
print(all_tickers)

['NAKA', 'BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL', 'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI', 'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'SUPX', 'TWG', 'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA', 'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ', 'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB', 'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL', 'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA']


In [40]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(all_tickers):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365, 252]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # Calculate Sharpe ratio
  historyPrices['Sharpe'] = (historyPrices['growth_252d'] - 0.045) / historyPrices['volatility']


  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 NAKA
1 BOW
2 HDL
3 RFAI
4 JDZG
5 RAY
6 BTOC
7 ZK
8 GPAT
9 PAL
10 SVCO
11 NNE
12 CCIX
13 VIK
14 ZONE
15 LOAR
16 MRX
17 RBRK
18 NCI
19 MFI
20 YYGH
21 TRSG
22 CDTG
23 CTRI
24 IBTA
25 MTEN
26 SUPX
27 TWG
28 ULS
29 PACS
30 MNDR
31 CTNM
32 MAMO
33 ZBAO
34 BOLD
35 MMA
36 UBXG
37 IBAC
38 AUNA
39 BKHA
40 LOBO
41 RDDT
42 ALAB
43 INTJ
44 RYDE
45 LGCL
46 SMXT
47 VHAI
48 DYCQ
49 CHRO
50 UMAC
51 HLXB
52 MGX
53 TBBB
54 TELO
55 KYTX
56 PMNT
57 AHR
58 LEGT
59 ANRO
60 GUTS
61 AS
62 FBLG
63 AVBP
64 BTSG
65 HAO
66 CGON
67 YIBO
68 JL
69 SUGP
70 JVSA
71 KSPI
72 CCTG
73 PSBD
74 SYNX
75 SDHC
76 ROMA


In [41]:
stocks_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_365d,growth_252d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
0,4.00,4.20,2.80,3.02,440600,0.0,0.0,NAKA,2024,5,...,NaN,NaN,0.784768,NaN,NaN,0,0.463576,NaN,0,NaN
1,2.99,3.11,2.35,2.66,147300,0.0,0.0,NAKA,2024,6,...,NaN,NaN,0.883459,NaN,NaN,0,0.285714,NaN,0,NaN
2,2.53,3.11,2.41,2.92,73800,0.0,0.0,NAKA,2024,6,...,NaN,NaN,0.955479,NaN,NaN,0,0.239726,NaN,0,NaN
3,2.91,3.09,2.60,2.73,51100,0.0,0.0,NAKA,2024,6,...,NaN,NaN,1.051282,NaN,NaN,0,0.179487,NaN,1,NaN
4,2.94,2.94,2.41,2.69,56500,0.0,0.0,NAKA,2024,6,...,NaN,NaN,1.111524,NaN,NaN,0,0.197026,NaN,1,NaN


In [47]:
# Convert 'Date' column to datetime if it's not already
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

# Filter the DataFrame for the trading day '2025-06-06'
filtered_stocks_df = stocks_df[stocks_df['Date'] == '2025-06-06']

# Display the head of the filtered DataFrame
display(filtered_stocks_df.head())

# Display the number of rows in the filtered DataFrame
#print(f"Number of rows in the filtered DataFrame: {len(filtered_stocks_df)}")

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_365d,growth_252d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
254,16.260000,17.200001,15.550000,15.880000,431200,0.0,0.0,NAKA,2025,6,...,NaN,5.438356,NaN,20.5610,17.24800,1,0.103904,136.860031,0,0.039408
521,36.439999,36.540001,35.930000,36.389999,137900,0.0,0.0,BOW,2025,6,...,NaN,1.442331,NaN,36.9540,37.70400,0,0.016763,24.508713,0,0.057014
792,20.430000,20.440001,19.969999,20.410000,1300,0.0,0.0,HDL,2025,6,...,NaN,1.007155,NaN,19.9660,21.08215,0,0.023028,18.816306,0,0.051134
1031,10.510000,10.510000,10.510000,10.510000,0,0.0,0.0,RFAI,2025,6,...,NaN,NaN,NaN,10.4930,10.47935,1,0.000000,0.356043,0,NaN
1304,0.380000,0.386000,0.201000,0.295000,3428900,0.0,0.0,JDZG,2025,6,...,NaN,0.168571,NaN,0.4883,0.52880,0,0.627119,1.475159,0,0.083768


In [50]:
filtered_stocks_df[['growth_252d','Sharpe']].describe()

,growth_252d,Sharpe
count,73.000000,73.000000
mean,1.227946,0.284576
std,1.480238,0.512601
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.083768
75%,1.446667,0.291048
max,8.097413,2.835668


Median sharpe ratio is ~ 0.08

In [62]:
# Calculate future growth columns
for i in range(1, 13):
    days = i * 21  # Assuming 1 month = 21 trading days
    stocks_df[f'future_growth_{i}m'] = stocks_df['Close'].shift(-days) / stocks_df['Close']

# Display the head of the DataFrame with the new columns
display(stocks_df.head())

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,4.00,4.20,2.80,3.02,440600,0.0,0.0,NAKA,2024,5,...,0.397351,0.350993,0.341060,0.387417,0.410596,0.397351,0.586093,0.516556,0.897351,5.672185
1,2.99,3.11,2.35,2.66,147300,0.0,0.0,NAKA,2024,6,...,0.439850,0.390226,0.379699,0.428571,0.529323,0.454887,0.642857,0.556391,1.150376,6.022556
2,2.53,3.11,2.41,2.92,73800,0.0,0.0,NAKA,2024,6,...,0.434931,0.349315,0.332192,0.541096,0.523973,0.414384,0.565068,0.489726,1.058219,5.438356
3,2.91,3.09,2.60,2.73,51100,0.0,0.0,NAKA,2024,6,...,0.487179,0.378388,0.340659,0.648352,0.511722,0.449084,0.564103,0.597070,1.384615,4.890110
4,2.94,2.94,2.41,2.69,56500,0.0,0.0,NAKA,2024,6,...,0.496283,0.405204,0.334572,0.620818,0.609665,0.464684,0.527881,0.650558,1.449814,4.312268


Question 3

In [63]:
# Determine the first trading day for each ticker
min_date_per_ticker = stocks_df.groupby('Ticker')['Date'].min().reset_index()

# Rename the column for clarity
min_date_per_ticker.rename(columns={'Date': 'min_day'}, inplace=True)

# Display the result
display(min_date_per_ticker.head())

# Display the info to see the data types
min_date_per_ticker.info()

,Ticker,min_day
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Ticker   77 non-null     object        
 1   min_day  77 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 1.3+ KB


In [64]:
# Perform an inner join between stocks_df (with future growth) and min_date_per_ticker
# The join is on Ticker and the first trading day
ipos_first_day_growth = pd.merge(
    stocks_df,
    min_date_per_ticker,
    left_on=['Ticker', 'Date'],
    right_on=['Ticker', 'min_day'],
    how='inner'
)

# Drop the redundant 'first_trading_day' column and keep relevant columns
future_growth_df = ipos_first_day_growth[['Ticker', 'Date'] + [f'future_growth_{i}m' for i in range(1, 13)]]


# Display the head and info of the resulting DataFrame
display(future_growth_df.head())
future_growth_df.info()

# Verify the number of records
print(f"\nNumber of records after join: {len(future_growth_df)}")

,Ticker,Date,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,NAKA,2024-05-31,0.728477,0.552980,0.397351,0.350993,0.341060,0.387417,0.410596,0.397351,0.586093,0.516556,0.897351,5.672185
1,BOW,2024-05-23,1.024790,1.139076,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
2,HDL,2024-05-17,0.775236,0.750112,0.673845,0.643786,0.720502,0.719605,1.255271,1.106326,1.030507,1.192014,0.907133,1.002602
3,RFAI,2024-07-05,1.002191,1.006972,1.007968,1.014940,1.014940,1.017928,1.022908,1.030877,1.035857,1.040837,1.046813,0.174303
4,JDZG,2024-05-15,0.242998,0.205160,0.122850,0.160442,0.272727,0.199017,0.139066,0.182555,0.170025,0.146437,0.121867,0.142506


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Ticker             77 non-null     object        
 1   Date               77 non-null     datetime64[ns]
 2   future_growth_1m   77 non-null     float64       
 3   future_growth_2m   77 non-null     float64       
 4   future_growth_3m   77 non-null     float64       
 5   future_growth_4m   77 non-null     float64       
 6   future_growth_5m   77 non-null     float64       
 7   future_growth_6m   77 non-null     float64       
 8   future_growth_7m   77 non-null     float64       
 9   future_growth_8m   77 non-null     float64       
 10  future_growth_9m   77 non-null     float64       
 11  future_growth_10m  77 non-null     float64       
 12  future_growth_11m  77 non-null     float64       
 13  future_growth_12m  77 non-null     float64       
dtypes: datetime6

In [65]:
# Select the future growth columns
future_growth_columns = [f'future_growth_{i}m' for i in range(1, 13)]

# Compute descriptive statistics for these columns
descriptive_stats = future_growth_df[future_growth_columns].describe()

# Display the descriptive statistics
display(descriptive_stats)

,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,0.926390,0.936866,0.834120,0.822711,0.802881,0.864347,0.846270,0.829971,0.878868,12.276241,12.293432,12.385054
std,0.342767,0.568481,0.410362,0.401473,0.486716,0.649114,0.706456,0.754255,0.925133,99.713681,100.009929,99.802283
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.048274,0.043103,0.033144,0.041357,0.023674,0.038947
25%,0.775236,0.680193,0.508139,0.515385,0.445378,0.387417,0.324100,0.225974,0.232468,0.251515,0.271143,0.229851
50%,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.777871,0.756546,0.752083
75%,1.047796,1.139076,1.075676,1.125316,1.017822,1.103896,1.150909,1.074739,1.037624,1.203535,1.326165,1.472441
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,875.862067,878.448251,876.724156


Answer 3 : from above oututs it seems its 2 months but may be wrong.

Question 4

In [66]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=c9ba4567-10a9-48d8-9646-6be9f906e1bb
To: /content/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 111MB/s]


In [67]:
df.head()

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [69]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [70]:
net_income

np.float64(24295.523125248386)